In [2]:
! apt install tesseract-ocr
! apt install libtesseract-dev
! pip install pytesseract

Reading package lists... Done
Building dependency tree       
Reading state information... Done
tesseract-ocr is already the newest version (4.00~git2288-10f4998a-2).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 42 not upgraded.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
libtesseract-dev is already the newest version (4.00~git2288-10f4998a-2).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 42 not upgraded.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
#import packages
import pytesseract
import cv2
from google.colab.patches import cv2_imshow
import numpy as np
import pandas as pd
import re
import os
import shutil
from google.colab import drive
import datetime


In [4]:
#this will connect to your Google Drive. It will ask you to allow access
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [4]:
# create a file list data frame to keep the old-name -> new-name conversions and status 
file_names_list = []

In [16]:
def unzip_src(folder):
  """
  Unzip zipped folder.

  A zipped folder is taken in and unzipped.

  Parameter
  ---------
  kind : Google Drive Folder
    Folder containing watershed photos.

  Returns
  -------
  new_src : new src for unzipped folder.
  unzipped : unzipped folder.

  """
  new_src = folder
  unzipped = None

  if folder[-4:] == '.zip':
    src_as_zip = True
    unzipped = folder[:-4]
    shutil.unpack_archive(folder, unzipped)

    parentZip = np.array(os.listdir(unzipped))
    new_src = unzipped + '/' + parentZip[0]
  
  return new_src, unzipped


def new_folder(src, dst = None):
  '''
  Create a new folder destination address for folder to be copied.

  This method's parameter is the address of folder you're going to copy, address of destination parent folder
  It will return the name of folder (to use later), full new address of copy folder 
  determines new folder address.

  Parameters
  ----------
  src : src address for folder to be copied.
  dst : destination address for folder to be copied.

  Returns
  -------
  folder_name : the new folders name.
  newAddress : the full new address of copy folder.
  '''
  if dst is None:
    dst = src
  # print(src, "Src")
  src_elements = src.split('/')
  folder_name = src_elements[-1][:2]
  # print(folder_name, "Folder")
  newAddress = dst + "/" + folder_name
  return folder_name, newAddress


def generate_picName(fdr_name, tStamp):
  '''
  Generate standardized picture names.

  The full source address of a pciture file and its time stamp are standardized 
  to a format delineated by water shed number and time stamp. 

  Parameters
  ----------
  fdr_name : full source address of current picture file.
  tStamp : unaltered timeStamp from bottom of the photo
      as a string.

  Returns
  -------
  new_name : new name of picture
  '''
  ws_num = fdr_name[1] #!!!this should be changed if src_elements[-2][1] will not be watershed number!!

  stamp_elements = re.split('[\n: -]', tStamp)
  date = stamp_elements[2] + stamp_elements[0] + stamp_elements[1]
  time = stamp_elements[3] + stamp_elements[4] + stamp_elements[5]

  new_name = "Hbwtr_w" + ws_num + '_' + date + '_' + time + '.JPG'
  return new_name

def extract_timeStamp(pic_address):
    '''
    Extract time Stamp from picture file.

    From the bottom right of each picture file, the time stamp is read as image using cv2. It is then converted to a string.
    text which is then checked for format and subsequently returned through match_date_format.

    Parameters
    ----------
    pic_address : full source address of current picture file.
    Returns
    -------
    match_date_format.group(0) : unaltered timeStamp from bottom of the photo as a string.
    '''
    # print(pic_address)
    img = cv2.imread(pic_address) #read as an image

    # check if the timestamp is the correct format
    date_pattern = "\d\d-\d\d-\d\d\d\d \d\d:\d\d:\d\d" # eg 12-12-2020 11:59:32

    loop = 5
    size_extension=0
    while loop>0:
      ts = img[2352 - size_extension:, 2000-size_extension:, :] #(change if sizing conventions change!)

      text = pytesseract.image_to_string(ts)
      match_date_format = re.search(date_pattern,text)
      if match_date_format:
        # found timestamp, return
        break

      size_extension+=100
      loop-=1

    if loop ==0:
      # reached end of loop without finding correct timestamp
      print("Correct timestamp not found")

    else:
      return match_date_format.group(0)

      

def rename_images(picFolder, fdr_name, fdr_dst):

  '''
  Rename all files in the current folder and populate the list of old and new names.

  For each picture file in the current folder, its filename is changed and its new name is generated in another method.
  The old image is copied to its new destination.

  Parameters
  ----------
  picFolder : a folder containing pictures.
  fdr_name : full address of folder.
  fdr_dst : full destination address.
  '''

  picFiles = np.array(os.listdir(picFolder))
  # print(picFiles, len(picFiles))
  old_folder = "/".join(picFolder.split("/")[-3:-1])

  for filename in picFiles:
    old_name = filename
    new_name = np.nan
    status="raw"
    note = np.nan
    print("Old filename", filename) #just to track where you are
    src = picFolder + '/' + filename #old img address
    # print(src)
    filetype = filename[-4:]
    try:
      if (filetype == '.JPG') or (filetype == '.jpg'):
        tStamp = extract_timeStamp(src)
        new_name = generate_picName(fdr_name, tStamp)
        print("New filename", new_name)
        status = "renamed"
        dst = fdr_dst + '/' + filename #new img address
        dst_renamed = fdr_dst + '/' + new_name #img new address + name
        shutil.copy(src, dst_renamed) #copies old image to new destination

        # os.rename(src, dst) #renames file in Google Drive
      else:
        print("Not image file")
        status= "error"
        note = "Not image file"
    except:
      print("Unable to process file")
      status = "error"
      note =  "Unable to process file"
    curr_list = [old_name, new_name, status, note, picFolder]
    file_names_list.append(curr_list)



In [6]:
# MAIN

from glob import glob

#collect all folder paths from newly uploaded data on folder
folder_list = glob("/content/drive/MyDrive/2_Camera Trap photos/COPY of data for script/Newly_uploaded_data/*/", recursive = True)
# collect all folder path from on deck folder
folder_list.extend(glob("/content/drive/MyDrive/2_Camera Trap photos/COPY of data for script/On_Deck/*/", recursive = True))

# extract folder_name 
folder_list = [f[:-1] for f in folder_list]

# for each folder rename and add them to the new destination - dst
for folder in folder_list:
  dst = "/content/drive/MyDrive/2_Camera Trap photos/COPY of data for script/Testing destination"
  save_as_zip = False

  #will unzip if necessary
  folder, unzipped = unzip_src(folder)

  # #create new destination folder
  fdr_name, fdr_dst = new_folder(folder, dst)

  if os.path.exists(fdr_dst):
    print("path already exists")
  else:
    print("new path")
    os.mkdir(fdr_dst)

  rename_images(folder, fdr_name, fdr_dst)

Streaming output truncated to the last 5000 lines.

New filename Hbwtr_w6_20211031_115851.JPG
Old filename 11010365.JPG
11-01-2021 12:03:48

New filename Hbwtr_w6_20211101_120348.JPG
Old filename 11020366.JPG
11-02-2021 11:58:44

New filename Hbwtr_w6_20211102_115844.JPG
Old filename 11030368.JPG
11-03-2021 11:58:41

New filename Hbwtr_w6_20211103_115841.JPG
Old filename 11040370.JPG
11-04-2021 11:58:38

New filename Hbwtr_w6_20211104_115838.JPG
Old filename 11050372.JPG
11-05-2021 11:58:35

New filename Hbwtr_w6_20211105_115835.JPG
Old filename 11060374.JPG
11-06-2021 11:58:32

New filename Hbwtr_w6_20211106_115832.JPG
Old filename 11070376.JPG
11-07-2021 11:58:28

New filename Hbwtr_w6_20211107_115828.JPG
Old filename 11080378.JPG
11-08-2021 11:58:25

New filename Hbwtr_w6_20211108_115825.JPG
Old filename 11090380.JPG
11-09-2021 11:58:22

New filename Hbwtr_w6_20211109_115822.JPG
Old filename 11100382.JPG
11-10-2021 11:58:19

New filename Hbwtr_w6_20211110_115819.JPG
Old f

In [7]:
# file_list data frame
file_df = pd.DataFrame(file_names_list, columns = ["old_name", "new_name","status", "note","old_folder"])
file_df

,old_name,new_name,status,note,old_folder
0,05100002.JPG,Hbwtr_w6_20210510_115951.JPG,renamed,NaN,/content/drive/MyDrive/2_Camera Trap photos/CO...
1,05110004.JPG,Hbwtr_w6_20210511_115948.JPG,renamed,NaN,/content/drive/MyDrive/2_Camera Trap photos/CO...
2,05120006.JPG,Hbwtr_w6_20210512_115945.JPG,renamed,NaN,/content/drive/MyDrive/2_Camera Trap photos/CO...
3,05130009.JPG,Hbwtr_w6_20210513_115941.JPG,renamed,NaN,/content/drive/MyDrive/2_Camera Trap photos/CO...
4,05140011.JPG,Hbwtr_w6_20210514_115938.JPG,renamed,NaN,/content/drive/MyDrive/2_Camera Trap photos/CO...
...,...,...,...,...,...
1423,03190008.JPG,Hbwtr_w2_20200319_115953.JPG,renamed,NaN,/content/drive/MyDrive/2_Camera Trap photos/CO...
1424,03220014.JPG,Hbwtr_w2_20200322_115945.JPG,renamed,NaN,/content/drive/MyDrive/2_Camera Trap photos/CO...
1425,03160002.JPG,Hbwtr_w2_20200316_120001.JPG,renamed,NaN,/content/drive/MyDrive/2_Camera Trap photos/CO...
1426,03170004.JPG,Hbwtr_w2_20200317_115959.JPG,renamed,NaN,/content/drive/MyDrive/2_Camera Trap photos/CO...


In [26]:
# files with error status
file_df[file_df.status=="error"]

,old_name,new_name,status,note,old_folder
211,desktop.ini,NaN,error,Not image file,/content/drive/MyDrive/2_Camera Trap photos/CO...
213,desktop.ini,NaN,error,Not image file,/content/drive/MyDrive/2_Camera Trap photos/CO...
374,10250350.JPG,NaN,error,Unable to process file,/content/drive/MyDrive/2_Camera Trap photos/CO...
375,desktop.ini,NaN,error,Not image file,/content/drive/MyDrive/2_Camera Trap photos/CO...
531,desktop.ini,NaN,error,Not image file,/content/drive/MyDrive/2_Camera Trap photos/CO...
532,desktop.ini,NaN,error,Not image file,/content/drive/MyDrive/2_Camera Trap photos/CO...
533,desktop.ini,NaN,error,Not image file,/content/drive/MyDrive/2_Camera Trap photos/CO...
1153,desktop.ini,NaN,error,Not image file,/content/drive/MyDrive/2_Camera Trap photos/CO...
1154,desktop.ini,NaN,error,Not image file,/content/drive/MyDrive/2_Camera Trap photos/CO...
1155,desktop.ini,NaN,error,Not image file,/content/drive/MyDrive/2_Camera Trap photos/CO...


In [19]:
# files unable to process
(file_df[file_df.note=="Unable to process file"][["old_folder", "old_name"]]).values

array([['/content/drive/MyDrive/2_Camera Trap photos/COPY of data for script/Newly_uploaded_data/W3 GC Channel 5-10-21 thru 10-24-21',
        '10250350.JPG'],
       ['/content/drive/MyDrive/2_Camera Trap photos/COPY of data for script/On_Deck/W2 GC Channel 2-5-19 thru 12-31-19',
        '10120510.JPG']], dtype=object)

In [21]:
# View images not processed
from os.path import exists
from IPython.display import Image
path = '/content/drive/MyDrive/2_Camera Trap photos/COPY of data for script/On_Deck/W2 GC Channel 2-5-19 thru 12-31-19/10120510.JPG'
file_exists  = exists(path)
print("file_exists:" , file_exists)
Image(path)

file_exists: True


In [22]:
path = "/content/drive/MyDrive/2_Camera Trap photos/COPY of data for script/Newly_uploaded_data/W1 GC Channel 11-7-20  thru 5-9-21/01130140.JPG"
file_exists = exists(path)
print("file_exists:" , file_exists)
Image(path)

file_exists: False
